# Nested Chats for Solving Complex Tasks with a Sequence of Inner Monologue

This notebook showcases how to use nested chats in AutoGen to to solving complex task with one or a sequence chats nested.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

### Example Tasks

In [9]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list}

tasks = [
    """On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.""",
    """Make a pleasant joke about it.""",
]

Since the first task could be complex to solve, we want to construct additional agents as an inner monologue, i.e., nested chats.

### Step 1. Agent Construction

#### Construction of a Group Chat to Be Used as an Inner Monologue
In the following code block, we construct a group chat manager which manages an inner_assistant agent and an inner_code_interpreter agent.


In [10]:
inner_assistant = autogen.AssistantAgent(
    "Inner-assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

inner_code_interpreter = autogen.UserProxyAgent(
    "Inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[inner_assistant, inner_code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
)

#### Construction of Individual Agents to Be Used in the Agent System



In [11]:
assistant_1 = autogen.AssistantAgent(
    name="Assistant_1",
    llm_config={"config_list": config_list},
    # is_termination_msg=lambda x: x.get("content", "") == "",
)

assistant_2 = autogen.AssistantAgent(
    name="Assistant_2",
    llm_config={"config_list": config_list},
    # is_termination_msg=lambda x: x.get("content", "") == "",
)
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    """,
)

reviewer = autogen.AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a compliance reviewer, known for your thoroughness and commitment to standards.
    Your task is to scrutinize content for any harmful elements or regulatory violations, ensuring
    all materials align with required guidelines.
    You must review carefully, identify potential issues, and maintain the integrity of the organization.
    Your role demands fairness, a deep understanding of regulations, and a focus on protecting against
    harm while upholding a culture of responsibility.
    """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

### Step 2: Orchestrate a Multi-Agent System with Nested Chats to Solve Tasks

In the following code block, at the outer level, we have communication between:

- user - assistant_1 for solving the first task, i.e., tasks[0].
- user - assistant_2 for solving the second task, i.e., tasks[1].

Since the first task is quite complicated, we created a sequence of nested chats as the inner monologue of Assistant_1.

- Nested Chat 1: assistant_1 - manager. This chat intends to delegate the task received by Assistant_1 to the Manager to solve.

- Nested Chat 2: assistant_1 - writer. This chat takes the output from Nested Chat 1, i.e., Assistant_1 vs. Manager, and lets the Writer polish the content to make an engaging and nicely formatted blog post, which is realized through the writing_message function.

- Nested Chat 3: assistant_1 - reviewer. This chat takes the output from Nested Chat 2 and intends to let the Reviewer agent review the content from Nested Chat 2.

- Nested Chat 4: assistant_1 - writer. This chat takes the output from previous nested chats and intends to let the Writer agent finalize a blog post.

The sequence of nested chats can be realized with the `register_nested_chats` function, which allows one to register one or a sequence of chats to a particular agent (in this example, the `assistant_1` agent).

Information about the sequence of chats can be specified in the `chat_queue` argument of the `register_nested_chats` function. The following fields are especially useful:
- `recipient` (required) specifies the nested agent;
- `message` specifies what message to send to the nested recipient agent. In a sequence of nested chats, if the `message` field is not specified, we will use the last message the registering agent received as the initial message in the first chat and will skip any subsequent chat in the queue that does not have the `message` field. You can either provide a string or define a callable that returns a string.
- `summary_method` decides what to get out of the nested chat. You can either select from existing options including "last_msg" and "reflection_with_llm", or or define your own way on what to get from the nested chat with a Callable.
- `max_turns` determines how many turns of conversation to have between the concerned agent pairs.

In [12]:
def writing_message(recipient, messages, sender, config):
    return f"Polish the content to make an engaging and nicely formatted blog post. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


nested_chat_queue = [
    {"recipient": manager, "summary_method": "reflection_with_llm"},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
    {"recipient": reviewer, "message": "Review the content provided.", "summary_method": "last_msg", "max_turns": 1},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
]
assistant_1.register_nested_chats(
    [autogen.Agent, None],
    nested_chat_queue,
)
# user.initiate_chat(assistant, message=tasks[0], max_turns=1)

res = user.initiate_chats(
    [
        {"recipient": assistant_1, "message": tasks[0], "max_turns": 1, "summary_method": "last_msg"},
        {"recipient": assistant_2, "message": tasks[1]},
    ]
)


********************************************************************************
Start a new chat with the following message: 
On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.

With the following carryover: 


********************************************************************************
User (to Assistant_1):

On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.

With the following carryover: 


********************************************************************************
Assistant_1 (to chat_manager):

On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.


/Users/qingyunwu/Documents/github/autogen/autogen/agentchat/chat.py:79: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(



>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is python)...
Inner-code-interpreter (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

Date: 2024-01-24, Close: $401.82
Date: 2024-01-25, Close: $404.12
Date: 2024-01-26, Close: $403.18
Date: 2024-01-29, Close: $408.96
Date: 2024-01-30, Close: $407.84
Date: 2024-02-01, Close: $403.03
Date: 2024-02-02, Close: $410.46
Date: 2024-02-05, Close: $404.90
Date: 2024-02-06, Close: $404.74
Date: 2024-02-07, Close: $413.29
Date: 2024-02-08, Close: $413.35
Date: 2024-02-09, Close: $419.77
Date: 2024-02-12, Close: $414.49
Date: 2024-02-13, Close: $405.57
Date: 2024-02-14, Close: $409.49
Date: 2024-02-15, Close: $406.56
Date: 2024-02-16, Close: $404.06
Date: 2024-02-20, Close: $402.79
Date: 2024-02-21, Close: $400.38

Microsoft stock was higher than $400 on 19 days between 2024-01-24 and 2024-02-21 in 2024.


--------------------------------------------------------------------------------
Inner-assistant (to chat_manager):

The 